In [1]:
import random

In [2]:
gameMap = ['go', 'cheap1', 'community', 'cheap3', 'parking', 'railroad','cheap6','chance','cheap8','medium9','jail',
       'medium11','utility','medium13','medium14','railroad','medium16','community','medium18','medium19','parking',
       'medium21','chance','medium23','medium24','railroad','medium26','medium27','utility','medium29','go to jail',
       'expensive31','expensive32','community','expensive34','railroad','chance','expensive37','parking','expensive39']

In [3]:
gameDict = {'go': [0], 'community': [2], 'parking': [0], 'railroad': [0], 'chance': [3], 'jail' : [0], 'go to jail':[4], 'utility' : [0],
           'cheap1':[1, 60, 10],'cheap3':[1, 60, 20],'cheap6':[1, 100,30],'cheap8':[1, 100, 30],
            'medium9':[1, 120, 40],'medium11':[1, 140, 50],'medium13':[1, 140, 50],'medium14':[1, 160, 60],'medium16':[1, 180, 70],
            'medium18':[1, 180, 70],'medium19':[1, 200, 80],'medium21':[1, 220, 90],'medium23':[1, 220, 90],'medium24':[1, 240, 100],
            'medium26':[1, 260, 110],'medium27':[1, 260, 110],'medium29':[1, 280, 120],
            'expensive31':[1, 300, 130],'expensive32':[1, 300, 130],'expensive34':[1, 320, 150],'expensive37':[1, 350, 175],'expensive39':[1, 400, 200]}

In [4]:
railRoad = [5, 15, 25 ,35]
debugSw = False

In [5]:
#id, position, debit, playerType 1: 一直买， 2：只买expensive
player1 = [1, 0, 500, 1]
player2 = [2, 0, 500, 2]
playerList = [[0, 0 ,99999], player1, player2] #第一个是无效用户，ignore
housePurchase = [0] * 40

In [6]:
def diceOnce():
    return int(6 * random.random() + 1)

def debugPrint(Text1, Text2):
    if(debugSw):
        print(Text1, Text2)

### Purchase strategies

In [ ]:
# purchase strategies, based on cost of tile

def noBuy(value):
    return False

def buyCheap(value):
    if(value <= 100):
        return True
    return False

def buyMedium(value):
    if (value <= 280 and value > 100):
        return True
    return False

def buyExpensive(value):
    if (value > 280):
        return True
    return False

def buyCheapAndExpensive(value):
    if (value <= 100 or value > 280):
        return True
    return False

def buyCheapAndMedium(value):
    if (value <= 100  or (value <= 280 and value > 100)):
        return True
    return False

def buyExpensiveAndMedium(value):
    if (value > 280 or (value <= 280 and value > 100)):
        return True
    return False

In [8]:
strategy = [noBuy, buyCheap, buyMedium, buyExpensive,buyCheapAndExpensive,buyCheapAndMedium,buyExpensiveAndMedium]

In [9]:
def purchase(player):
    debugPrint("------------", "--")
    debugPrint("beforePurchase:", player)
    pos = player[1]
    value = gameDict[gameMap[pos]]
    posType = value[0]
    if posType != 1:
        return
    if(player[2] <= value[1]):
        return
    if(not strategy[player[3]](value[1])):
        return
    player[2] = player[2] - value[1]
    housePurchase[pos] = player[0]
    debugPrint("AfterPurchase:", player)

In [10]:
def needRent(player):
    pos = player[1]
    return housePurchase[pos] != 0 and housePurchase[pos] != player[0]

In [11]:
def rent(player):
    debugPrint("------------", "--")
    debugPrint("beforeRent:", player)
    pos = player[1]
    toPlayerId = housePurchase[pos]
    toPlayer = playerList[toPlayerId]
    rentValue = min(gameDict[gameMap[pos]][2], player[2])
    toPlayer[2] += rentValue;
    player[2] -=rentValue;
    debugPrint("Rent:", rentValue)
    debugPrint("AfterRent:", player)


In [12]:
def chance(player):
    debugPrint("------------", "--")
    chance_destiny_pos = [0, 7, 10, 11, 24, 39]
    utility = [12, 28]
    randValue = int(16 * random.random() + 1)
    debugPrint("beforeChance",player)
    debugPrint('randValue', randValue)
    pos = player[1]
    if(randValue <= 2):
        nearestRailPos = 0
        nearestValue = 9999
        for i in range(len(railRoad)):
            dist = abs(railRoad[i]-pos)
            if(dist<nearestValue):
                nearestRailPos = railRoad[i]
                nearestValue = dist
        player[1] = nearestRailPos
        debugPrint("afterChance",player)
        return
    if(randValue <= 3):
        player[1] -= 3
        debugPrint("afterChance",player)
        return
    if(randValue <= 4):
        nearestUtilityPos = 0
        nearestValue = 9999
        for i in range(len(utility)):
            dist = abs(utility[i]-pos)
            if(dist<nearestValue):
                nearestUtilityPos = utility[i]
                nearestValue = dist
        player[1] = nearestUtilityPos
        debugPrint("afterChance",player)
        return
    if(randValue <= 10):
        debugPrint("afterChance",player)
        return
    player[1]=chance_destiny_pos[randValue-11]
    debugPrint("afterChance",player)

In [13]:
def community(player):
    debugPrint("------------", "--")
    debugPrint("beforeCommunity",player)
    community_destiny_pos = [0, 10]
    randValue = int(16 * random.random() + 1)
    debugPrint("randValue",randValue)
    if (randValue <= 2):
        player[1]=community_destiny_pos[randValue-1]
    debugPrint("afterCommunity",player)

In [14]:
def game(MaxRound = 999999):
    curPlayerId = 1
    roundNumber = 0
    while(MaxRound > 0):
        MaxRound -= 1
        roundNumber += 1
        end = False
        for i in range(len(playerList)):
            if(playerList[i][2] <= 0):
                end = True
                print('result', playerList)
                return playerList[i][0];
        t1 = diceOnce()
        t2 = diceOnce()
        move = t1 + t2
        curPlayer = playerList[curPlayerId]
        posBeforeMove = curPlayer[1]
        curPos = posBeforeMove + move
        curPos = curPos % 40
        debugPrint("------------", "--")
        debugPrint('roundNumber', roundNumber)
        debugPrint('current PlayerId',curPlayerId)
        debugPrint('curent Position',curPos)
        debugPrint('curent Position Type',gameMap[curPos])
        posType = gameDict[gameMap[curPos]][0]
        if(posType == 0):
            playerList[curPlayerId][1] = curPos
            curPlayerId = curPlayerId + 1
            if(curPlayerId == len(playerList)):
                curPlayerId = 1
            continue
        if(posType == 4):
            playerList[curPlayerId][1] = 10
            curPlayerId = curPlayerId + 1
            if(curPlayerId == len(playerList)):
                curPlayerId = 1
            continue
        if(posType == 1):
            playerList[curPlayerId][1] = curPos
            if(housePurchase[curPos] == 0):
                purchase(playerList[curPlayerId])
                curPlayerId = curPlayerId + 1
                if(curPlayerId == len(playerList)):
                    curPlayerId = 1
                continue
            if(needRent(playerList[curPlayerId])):
                rent(playerList[curPlayerId])
                curPlayerId = curPlayerId + 1
                if(curPlayerId == len(playerList)):
                    curPlayerId = 1
                continue
        if(posType == 2):
            playerList[curPlayerId][1] = curPos
            community(playerList[curPlayerId])
            curPlayerId = curPlayerId + 1
            if(curPlayerId == len(playerList)):
                curPlayerId = 1
            continue
        if(posType == 3):
            playerList[curPlayerId][1] = curPos
            chance(playerList[curPlayerId])
            curPlayerId = curPlayerId + 1
            if(curPlayerId == len(playerList)):
                curPlayerId = 1
            continue

In [15]:
#id, position, debit, playerType 1: 一直买， 2：只买expensive
player1 = [1, 0, 500, 1]
player2 = [2, 0, 500, 2]
playerList = [[0, 0 ,99999], player1, player2]
housePurchase = [0] * 40
debugSw = True

In [ ]:
game()

Probability of each tile landed

In [36]:
prob_player1 = [1, 0, 500, 0]
prob_player2 = [2, 0, 500, 0]
playerList = [[0, 0 ,99999], prob_player1, prob_player2]
housePurchase = [0] * 40
debugSw = False

In [37]:
count = [0]*40

In [38]:
def gameCountProb(MaxRound = 999999):
    curPlayerId = 1
    roundNumber = 0
    while(MaxRound > 0):
        MaxRound -= 1
        roundNumber += 1
        end = False
        for i in range(len(playerList)):
            if(playerList[i][2] <= 0):
                end = True
                break
        if(end):
            break
        t1 = diceOnce()
        t2 = diceOnce()
        move = t1 + t2
        curPlayer = playerList[curPlayerId]
        posBeforeMove = curPlayer[1]
        curPos = posBeforeMove + move
        curPos = curPos % 40
        debugPrint("------------", "--")
        debugPrint('roundNumber', roundNumber)
        debugPrint('current PlayerId',curPlayerId)
        debugPrint('curent Position',curPos)
        debugPrint('curent Position Type',gameMap[curPos])
        posType = gameDict[gameMap[curPos]][0]
        if(posType == 0):
            playerList[curPlayerId][1] = curPos
            curPlayerId = curPlayerId + 1
            if(curPlayerId == len(playerList)):
                curPlayerId = 1
            count[playerList[curPlayerId][1]] += 1;
            continue
        if(posType == 4):
            playerList[curPlayerId][1] = 10
            curPlayerId = curPlayerId + 1
            if(curPlayerId == len(playerList)):
                curPlayerId = 1
            count[playerList[curPlayerId][1]] += 1;
            continue
        if(posType == 1):
            playerList[curPlayerId][1] = curPos
            if(housePurchase[curPos] == 0):
                purchase(playerList[curPlayerId])
                curPlayerId = curPlayerId + 1
                if(curPlayerId == len(playerList)):
                    curPlayerId = 1
                count[playerList[curPlayerId][1]] += 1;
                continue
            if(needRent(playerList[curPlayerId])):
                rent(playerList[curPlayerId])
                curPlayerId = curPlayerId + 1
                if(curPlayerId == len(playerList)):
                    curPlayerId = 1
                count[playerList[curPlayerId][1]] += 1;
                continue
        if(posType == 2):
            playerList[curPlayerId][1] = curPos
            community(playerList[curPlayerId])
            curPlayerId = curPlayerId + 1
            if(curPlayerId == len(playerList)):
                curPlayerId = 1
            count[playerList[curPlayerId][1]] += 1;
            continue
        if(posType == 3):
            playerList[curPlayerId][1] = curPos
            chance(playerList[curPlayerId])
            curPlayerId = curPlayerId + 1
            if(curPlayerId == len(playerList)):
                curPlayerId = 1
            count[playerList[curPlayerId][1]] += 1;
            continue
    debugPrint('result', playerList)

In [39]:
gameCountProb(100000000)

In [40]:
print(count)

[3130877, 2186384, 1942125, 2228815, 2395433, 2569982, 2319685, 1346670, 2351764, 2336089, 5867589, 2739720, 2484318, 2408095, 2508033, 2658126, 2799283, 2579001, 2914669, 3042133, 2836072, 2788288, 1030674, 2699257, 3151967, 3026331, 2675932, 2650085, 2925048, 2562584, 0, 2660873, 2612589, 2380600, 2503368, 2734375, 871336, 2207352, 2207833, 2666645]


In [41]:
sum(count)

100000000

In [42]:
for i in range(len(count)):
    count[i] = count[i]/sum(count)
print(count)

[0.03130877, 0.022570494411241294, 0.020511922441268125, 0.024032782424884234, 0.02646542245166476, 0.029165773771895047, 0.027116108096662782, 0.01618074504034473, 0.028722070852364272, 0.029374323623924482, 0.07601274228339755, 0.038411999318943325, 0.036222538517166704, 0.0364307864554289, 0.03937723898304388, 0.0434444902087729, 0.04782949227988653, 0.04627918994604987, 0.05484060289722998, 0.0605600351886705, 0.06009746340334121, 0.06286279647894913, 0.024795576367357398, 0.06658884172539216, 0.08330400803066601, 0.08725198746602758, 0.08452459094404291, 0.09143682221208083, 0.11108084742212408, 0.10947678414033457, 0.0, 0.1276505880697228, 0.14367436668069591, 0.15288176160115416, 0.18977976663590101, 0.25584692072559856, 0.1095583492890323, 0.3116922069746692, 0.45293705878702734, 0.9999987773797112]


Test for different Strategy

In [46]:
#Comparing only Cheap & only Medium - First Mover only buy cheap
def init():
    global test_player1
    global test_player2
    global playerList
    global housePurchase
    global debugSw
    test_player1 = [1, 0, 500, 1] #only buyCheap
    test_player2 = [2, 0, 500, 2] #only buyMedium
    playerList = [[0, 0 ,99999], test_player1, test_player2]
    housePurchase = [0] * 40
    debugSw = False

In [ ]:
loseTime = [0] * 10
for i in range(10000):
    init()
    a = game()
    loseTime[a] += 1

In [48]:
loseTime

[0, 7089, 2911, 0, 0, 0, 0, 0, 0, 0]

In [49]:
#Comparing only Cheap & only Medium - First Mover only buy medium
def init():
    global test_player1
    global test_player2
    global playerList
    global housePurchase
    global debugSw
    test_player1 = [1, 0, 500, 1] #only buyCheap
    test_player2 = [2, 0, 500, 2] #only buyMedium
    playerList = [[0, 0 ,99999], test_player1, test_player2]
    housePurchase = [0] * 40
    debugSw = False

In [ ]:
loseTime = [0] * 10
for i in range(10000):
    init()
    a = game()
    loseTime[a] += 1

In [51]:
loseTime

[0, 7042, 2958, 0, 0, 0, 0, 0, 0, 0]

In [52]:
#Comparing only Cheap & only Expensive - First Mover only buy Cheap
def init():
    global test_player1
    global test_player2
    global playerList
    global housePurchase
    global debugSw
    test_player1 = [1, 0, 500, 1] #only buy Cheap
    test_player2 = [2, 0, 500, 3] #only buy Expensive
    playerList = [[0, 0 ,99999], test_player1, test_player2]
    housePurchase = [0] * 40
    debugSw = False

In [ ]:
loseTime = [0] * 10
for i in range(10000):
    init()
    a = game()
    loseTime[a] += 1

In [54]:
loseTime

[0, 7104, 2896, 0, 0, 0, 0, 0, 0, 0]

In [55]:
#Comparing only Cheap & only Expensive - First Mover only buy Cheap
def init():
    global test_player1
    global test_player2
    global playerList
    global housePurchase
    global debugSw
    test_player1 = [1, 0, 500, 3] #only buy Cheap
    test_player2 = [2, 0, 500, 1] #only buy Expensive
    playerList = [[0, 0 ,99999], test_player1, test_player2]
    housePurchase = [0] * 40
    debugSw = False

In [ ]:
loseTime = [0] * 10
for i in range(10000):
    init()
    a = game()
    loseTime[a] += 1

In [57]:
loseTime

[0, 2735, 7265, 0, 0, 0, 0, 0, 0, 0]